In [7]:
from pathlib import Path
import pandas as pd
import requests
import subprocess 

In [32]:
multifasta_proteins = '../../results/cassettes/clusters/merged_cassette_proteins.faa.gz'
clusters_df = '../../results/cassettes/clusters/mmseqs2_cluster.tsv'

In [31]:
def cassettes_headers_to_df(multifasta_proteins):
    cmd = ['zgrep', '-Po', '(?<=^\>).+', multifasta_proteins]
    columns = ['contigs', 'cassettes', 'locus_tags', 'header']
    data = []
    for header in subprocess.run(cmd, check=True, stdout=subprocess.PIPE).stdout.decode('UTF-8').splitlines():
        rows = header.split(':')
        assert len(rows) == 3, 'Header {} is invalid !'.format(header)
        rows.append(header)
        data.append(rows)
        
    df = pd.DataFrame(data=data, columns=columns)
    return df    

In [33]:
headers_df = cassettes_headers_to_df(multifasta_proteins=multifasta_proteins)
clusters_df = pd.read_csv(clusters_df, sep='\t', header=None).rename(columns={0:'representative', 1:'member'})

In [34]:
clusters_df.head()

,representative,member
0,Contig_936157.3_6:Cassette_1:IOMDNFNB_00323,Contig_936157.3_6:Cassette_1:IOMDNFNB_00323
1,Contig_936157.3_6:Cassette_1:IOMDNFNB_00323,Contig_90370.3085_12:Cassette_1:MFLNKJEA_02786
2,Contig_936157.3_6:Cassette_1:IOMDNFNB_00323,Contig_59201.1197_9:Cassette_1:MPAOKENP_03266
3,Contig_936157.3_6:Cassette_1:IOMDNFNB_00323,Contig_590.13417_5:Cassette_1:GBGIPKHN_02573
4,Contig_936157.3_6:Cassette_1:IOMDNFNB_00323,Contig_590.14226_3:Cassette_1:KDNHJPFP_01758
